In [1]:
from rapidfuzz import process, fuzz
import numpy
import pandas
import pathlib
import pydash
import requests

def value_extract(row, column):

    ''' Extract dictionary values. '''
    
    return pydash.get(row[column], 'value')

def sparql_query(query, service):

    ''' Send sparql request, and formulate results into a dataframe. '''

    response = requests.get(service, params={'format': 'json', 'query': query}, timeout=120)
    results = pydash.get(response.json(), 'results.bindings')
    df = pandas.DataFrame.from_dict(results)
    for column in df.columns:
        df[column] = df.apply(value_extract, column=column, axis=1)
    
    return df

def format_creators(row):

    creators = []

    if row['creators_primary'] is not numpy.nan:
        creators += row['creators_primary'].split(',')

    if row['creators_other'] is not numpy.nan:
        creators += row['creators_other'].split(',')
        
    if len(creators):
        return pydash.uniq([f'creators/{x}' for x in creators])
    else:
        return None

def creator_link_check(row, extant):

    extant_creator = [(x in extant) for x in row['creator_id']]
    return all(extant_creator)


def related_entities(row):
    creator_array = row['creator_id']
    creator_array = ' '.join([f'"{x}"' for x in creator_array])

    try:
        query = '''
            select distinct ?creators ?wikidata ?element ?elementLabel where {
                values ?creators {'''+creator_array+'''}
                ?wikidata wdt:P7003 ?creators .
                ?element ?p ?wikidata .
                ?element wdt:P31 ?elementType .
                service wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }
            } '''

        related_entities = sparql_query(query, 'https://query.wikidata.org/sparql').drop_duplicates()
        if len(related_entities):
            return (list(related_entities.elementLabel.unique()))
        else:
            return []
    except:
        return 'error'

def title_matching(row):

    c = process.extract(row['title'], row['entity_array'], scorer=fuzz.WRatio, limit=50)
    candidates = [x for x in c if x[1] > 85] 
    return candidates


def count_array(row):
    return len(row['title_matches'])
    
def max_match(row):

    if len(row['title_matches']):
        return max([x[1] for x in row['title_matches']])
    else:
        return 0
    

query = '''
    select ?acmi_id ?wikidata_id
    where {
        ?wikidata_id wdt:P7003 ?acmi_id .
        } '''

extant_links = sparql_query(query, 'https://query.wikidata.org/sparql').drop_duplicates()

print(len(extant_links))
extant_links.head()

14663


,wikidata_id,acmi_id
0,http://www.wikidata.org/entity/Q703727,creators/72597
1,http://www.wikidata.org/entity/Q172837,works/115714
2,http://www.wikidata.org/entity/Q6940276,works/70682
3,http://www.wikidata.org/entity/Q189694,creators/59635
4,http://www.wikidata.org/entity/Q221917,creators/77040


In [5]:
acmi_works = pandas.read_csv(pathlib.Path.cwd().parents[0] / 'acmi-api' / 'app' / 'tsv' / 'works.tsv', delimiter='\t', low_memory=False)
acmi_works = acmi_works.loc[acmi_works.record_type.isin(['work'])]

for x in ['[DVD]', '[Widescreen]', '[NTSC]', '[B&W]', '[Italian version]',
    '[Edited version]', '[Greek version]', '[study extract]', '[Dubbed]',
    '[Turkish version]', '[game trailer]', '[a discussion]']:
    acmi_works['title'] = acmi_works['title'].str.replace(x, '')

acmi_works = acmi_works.rename(columns={'id':'work_id'})
acmi_works['creator_id'] = acmi_works.apply(format_creators, axis=1)
acmi_works['work_id'] = 'works/'+acmi_works['work_id'].astype(str)
acmi_works = acmi_works[['work_id', 'title', 'type', 'creator_id']].dropna()
acmi_works = acmi_works.loc[~acmi_works.work_id.isin(list(extant_links.acmi_id))]
acmi_works['creator_linked'] = acmi_works.apply(creator_link_check, extant=list(extant_links.acmi_id), axis=1)
acmi_works = acmi_works.loc[acmi_works.creator_linked]

# sample group
acmi_works = acmi_works[:100]

# okay now work through one by one pulling concated filmographies and finding any vague matches to title
# then return two columns, match numbers and highest score.

acmi_works['entity_array'] = acmi_works.apply(related_entities, axis=1)
acmi_works = acmi_works.loc[~acmi_works.entity_array.isin(['error'])]
acmi_works['title_matches'] = acmi_works.apply(title_matching, axis=1)
acmi_works['candidate_count'] = acmi_works.apply(count_array, axis=1)
acmi_works['highest_match'] = acmi_works.apply(max_match, axis=1)

# acmi_works = acmi_works.loc[acmi_works.candidate_count == 0]

print(len(acmi_works))
acmi_works.head()

1944
100
75


,work_id,title,type,creator_id,creator_linked,entity_array,title_matches,candidate_count,highest_match
0,works/119934,The Dame Was Loaded German advertisement,Object,[creators/41813],True,"[Star Wars, Super Solitaire, Terrormolinos, Mu...","[(The Dame Was Loaded, 90.0, 17), (The Muncher...",7,90.0
29,works/99275,The road to total war,TV show,"[creators/76290, creators/86050]",True,"[Ryan, Strange Invaders, Rebels with a Camera,...","[(The Street, 85.5, 6), (The Big Snit, 85.5, 2...",33,85.5
50,works/64547,Adventure playground: London,Film,"[creators/11442, creators/11443]",True,"[Corinne Cantrill, Arthur Cantrill, Arthur and...",[],0,NaN
52,works/78485,Trapeze,Film,"[creators/20569, creators/12143]",True,"[Patrick Watson, Arthur Hiller, ARTV, Hubert A...",[],0,NaN
58,works/77749,Tennis club,Film,[creators/12329],True,"[Allegro Non Troppo, Category:Films directed b...",[],0,NaN


In [7]:
# well, these can be written, no?

# three categories: one is no matches (create), one strong match (link extant), multi (manual assess).

# cast all text to uppercase!

test = acmi_works.copy()
test = test.loc[test.candidate_count == 1]

print(len(test))
test.head()

8


,work_id,title,type,creator_id,creator_linked,entity_array,title_matches,candidate_count,highest_match
146,works/84797,Island,Film,[creators/20692],True,"[Q106717666, Calcutta, Careful, Vincent, Cactu...","[(Island, 100.0, 6)]",1,100.000000
344,works/119182,[John Robertson Duigan Film Collection: Reel 14],Film,[creators/20892],True,"[Virginia Duigan, Romero, Sirens, Molly, Paran...","[(Virginia Duigan, 85.5, 0)]",1,85.500000
577,works/65482,Blackwood,Film,"[creators/74188, creators/74145, creators/7392...",True,"[Barbara Ann Roberts, Les Drew, Reading Alista...","[(Blackwood, 100.0, 351)]",1,100.000000
1109,works/86382,Look who's laughing,Film,[creators/77600],True,"[The Actress and the Cowboys, The Animal, The ...","[(Look Who's Laughing, 89.47368421052632, 260)]",1,89.473684
1433,works/91465,Shirley Valentine,Film,"[creators/13548, creators/11230]",True,"[Viacom, French Postcards, Madagascar 3: Europ...","[(Shirley Valentine, 100.0, 138)]",1,100.000000
